In [72]:
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('beachcondos.db')

# Create a cursor object
cur = conn.cursor()

condos = pd.read_csv('Condo_Unit.csv')
client = pd.read_csv('Client.csv')
owner = pd.read_csv('Owner.csv')


# Drop the database tables the second time this runs
cur.execute('DROP TABLE IF EXISTS Condo_Unit')
cur.execute('DROP TABLE IF EXISTS Owner')
cur.execute('DROP TABLE IF EXISTS Client')
conn.commit()  # Commit the changes to the database

# Append the DataFrames into the tables
condos.to_sql('Condo_Unit', con=conn, if_exists='append', index=False)
owner.to_sql('Owner', con=conn, if_exists='append', index=False)
client.to_sql('Client', con=conn, if_exists='append', index=False)


In [65]:
print("(1)")
cur.execute('SELECT First_Name, Last_Name FROM Owner WHERE First_Name LIKE "%jo%"')
df1 = pd.DataFrame(cur.fetchall(), columns=['First Name', 'Last Name'])
df1

(1)


,First Name,Last Name
0,Joe,Heulbert
1,Mary Jo,Gonzalez


In [64]:
print("(2)")
cur.execute('SELECT Client_Name, State '
           'FROM Client '
           'WHERE State IN (\'NY\', \'FL\')')
df2 = pd.DataFrame(cur.fetchall(), columns=['Client_Name', 'State'])
df2

(2)


,Client_Name,State
0,John Alton,FL
1,Lucy Ferb,NY
2,Diana Perez,NY


In [75]:
#From the Client table: display the Names, Street and City of all clients from Colorado or
#Michigan whose last name contains a letter m

print("(3)")
cur.execute('SELECT Client_Name, Street, City, State '
           'FROM Client '
           'WHERE State IN (\'CO\', \'MI\') '
           'AND Client_Name LIKE \'%m%\'')
df3 = pd.DataFrame(cur.fetchall(), columns=['Client_Name', 'Street', 'City', 'State'])
df3


(3)


,Client_Name,Street,City,State
0,Jeff Miller,134 Central,Berridge,CO
1,Benton Smith,12 Mountain,Denton,CO
2,George Maun,76 Dixon,Mason,MI


In [76]:
#From the Client table: display the Names and the State of all clients from a state of your choice.
print("(4)")
cur.execute('SELECT Client_Name, State '
           'FROM Client '
           'WHERE State IN (\'MI\') ')
df4 = pd.DataFrame(cur.fetchall(), columns=['Client_Name', 'State'])
df4


(4)


,Client_Name,State
0,George Maun,MI


In [98]:
#From the Condo Unit table: display Weekly Rate, number of Bathrooms and Unit Number of all
#units with rates greater than or equal to $725 and less than $1000

print("(5)")
cur.execute('SELECT Weekly_Rate, Bathrooms, Unit_Number'
           'FROM Condo_Unit '
           'WHERE Weekly_Rate >= 725 AND Weekly_Rate < 1000')
df5 = pd.DataFrame(cur.fetchall(), columns=['Weekly_Rate', 'Bathrooms', 'Unit_Number'])
df5


(5)


OperationalError: no such column: Weekly_Rate

In [99]:
#From the Condo Unit table: display Unit Number, Bedrooms, Rented and Next Available of all
#two-bedroom units which are rented and also display Unit Number, Bedrooms, Rented and Next
#Available of three-bedroom units available after January 1st , 2023. Note: sqlite3 does not support
#Date type. With the date in this format, we can get the right comparison YYYY-MM-DD

print("(6)")
cur.execute('SELECT Unit_Number, Bedrooms, Rented, Next_Available '
           'FROM Condo_Unit '
           'WHERE Bedrooms = 2 AND Rented = 1 '
           'UNION '
           'SELECT Unit_Number, Bedrooms, Rented, Next_Available '
           'FROM Condo_Unit '
           'WHERE Bedrooms = 3 AND Next_Available > \'2023-01-01\'')
df6 = pd.DataFrame(cur.fetchall(), columns=['Unit Number', 'Bedrooms', 'Rented', 'Next Available'])
df6

(6)


,Unit Number,Bedrooms,Rented,Next Available
0,109,2,1,10/8/2023
1,205,2,1,8/15/2023
2,207,2,1,2/6/2023
3,511,2,1,3/11/2023


In [77]:
#From the Owner table: display the first name, last name, street, city and state of all owners
#whose last names start with the letter G and are from New York

print("(7)")
cur.execute('SELECT First_Name, Last_Name, Street, City, State '
           'FROM Owner '
           'WHERE State = \'NY\' '
           'AND Last_Name LIKE \'g%\'')
df7 = pd.DataFrame(cur.fetchall(), columns=['First Name', 'Last Name', 'Street', 'City', 'State'])
df7

(7)


,First Name,Last Name,Street,City,State
0,Mary Jo,Gonzalez,56 Central,Nowhere,NY


In [78]:
#From the Condo Unit table: display average weekly rates grouped by number of bedrooms
print("(8)")

cur.execute('SELECT Bedrooms, AVG(Weekly_Rate) AS "Average Weekly Rate" '
           'FROM Condo_Unit '
           'GROUP BY Bedrooms')
df8 = pd.DataFrame(cur.fetchall(), columns=['Bedrooms', 'Average Weekly Rate'])

df8

(8)


,Bedrooms,Average Weekly Rate
0,1,758.333333
1,2,1000.000000
2,3,1300.000000


In [89]:
#From all three tables display Client Name, Weekly Rate, Rented and Owner’s Last Name of all
#condos that are rented and whose owner’s last name starts with A.

print("(9)")

cur.execute('''SELECT c.Client_Name, cu.Weekly_Rate, cu.Rented, o.Last_Name '
           'FROM Client c 
            JOIN Condo_Unit cu ON c.Client_Number = cu.Client_Number
            JOIN Owner o ON cu.Owner_Code = o.Owner_Code
            WHERE cu.Rented = TRUE AND o.Last_Name LIKE "A%"''')
query = cur.fetchall()
df9 = pd.DataFrame(query, columns=['Client Name', 'Weekly Rate','Rented', "Owner Last Name"])
df9


(9)


,Client Name,Weekly Rate,Rented,Owner Last Name
0,John Alton,1450,1,Alonso
1,West Kachi,800,1,Alonso


In [92]:

print("(10)")
cur.execute('''SELECT c.City, c.State, cu.Bedrooms, cu.Rented, o.State as "Owner State"
           FROM Client c 
           JOIN Condo_Unit cu ON c.Client_Number = cu.Client_Number
           JOIN Owner o ON cu.Owner_Code = o.Owner_Code
           WHERE cu.Rented = 1 AND cu.Bedrooms = 2 AND o.State = 'CO' ''')

query = cur.fetchall()
df10 = pd.DataFrame(query, columns=['City', 'State', 'Bedrooms', 'Rented', 'Owner State'])
df10


(10)


,City,State,Bedrooms,Rented,Owner State
0,Berridge,CO,2,1,CO
1,Tarleton,CO,2,1,CO
